In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import time
from datetime import datetime, timedelta

all_currency_pairs = ['USDCHF10080',
                 'GBPUSD10080', 'EURUSD10080', 'USDJPY10080', 'USDCAD10080', 'AUDUSD10080', 'NZDUSD10080',
                 'GBPCHF10080', 'EURCHF10080', 'CHFJPY10080', 'CADCHF10080', 'AUDCHF10080', 'NZDCHF10080', 'EURGBP10080',
                 'GBPJPY10080', 'GBPCAD10080', 'GBPAUD10080', 'EURJPY10080', 'EURCAD10080', 'EURAUD10080', 'EURNZD10080',
                'CADJPY10080', 'AUDJPY10080', 'NZDJPY10080', 'AUDCAD10080', 'NZDCAD10080', 'AUDNZD10080'
                ]



headers = ['date', 'ignore', 'open', 'high', 'low', 'close', 'volume'  ]


In [ ]:
raw_df = pd.read_csv("../raw_weekly_data/NZDCAD10080.csv", names = headers )

needed_weeks_df = raw_df.loc[(raw_df['date'] >= "2015.01.01")   ]   

needed_weeks_df = needed_weeks_df.reset_index(drop=True)

available_weeks = needed_weeks_df['date'].values



#print(raw_df)
print(available_weeks)

In [ ]:
for y in available_weeks:
    
    current_date = datetime.strptime(y, '%Y.%m.%d').strftime('%A')
    
    if current_date != 'Sunday':
        available_weeks = [x for x in available_weeks if x != y]
        print(y)

In [ ]:
for a in available_weeks:
  
  
    path = "./files/" + a

    try: 
        os.makedirs(path)
        control_file = pd.DataFrame()
        
        for  b in range(8):
            control_file['control' + str(b)] = -1
            
            
        control_file.to_csv(path + '/control.csv')
        
        

    except OSError as error: 
         print(error)  
 

In [ ]:
all_currency_pair_df = []

for d in all_currency_pairs:
    
    current_currency_df = pd.read_csv("../raw_weekly_data/" + d + ".csv", names = headers ) 
    
    current_currency_df = current_currency_df.iloc[::-1]
    current_currency_df = current_currency_df.reset_index(drop=True)
      
    current_currency_df = {'name': d.replace('10080',''),'dataframe': current_currency_df }
    
    all_currency_pair_df.append(current_currency_df)
    

print(all_currency_pair_df)

    
    

In [ ]:
acceptable_signal_percent = 60

analysis_span = 52

analysis_interval = 3

no_of_weeks_analysis = [c for c in range(0, analysis_span, analysis_interval)]

all_time_next_week_trade =[]

for e in available_weeks:
    
    next_week_trade = []

    for f in all_currency_pair_df:
        
        
        current_currency_df = f['dataframe'].copy(deep = True)

        current_currency_df = current_currency_df.loc[(current_currency_df['date'] <= e )   ] 
        
        
        
        current_currency_df = current_currency_df.reset_index(drop=True)
        
        

        current_currency_df = current_currency_df.head(52)
        
        #print(f['name'], current_currency_df)

        needed_value_list = []
        
        needed_direction_list = []
        
        for g in no_of_weeks_analysis:
            
            needed_value = current_currency_df.copy()['close'][g]
            
            needed_value_list.append(needed_value)
            
            
        
        #print(f['name'], needed_value_list)
       
        
        for index, h in enumerate(needed_value_list):
            
            if index == 0:
                
                continue
                
            if h - needed_value_list[index-1] > 0 :
                
                needed_direction_list.append('S')
            
            else:
                
                needed_direction_list.append('B')
                
                
        #print(f['name'],needed_direction_list)  
        
        if (100 * needed_direction_list.count('S') / len(needed_direction_list) ) > acceptable_signal_percent:
            
            next_week_trade.append({'name': f['name'], 'direction': 'B'})
        
        if ( 100 * needed_direction_list.count('B') / len(needed_direction_list) ) > acceptable_signal_percent:
            
            next_week_trade.append({'name': f['name'], 'direction': 'S'})
    
    
    
    if len(next_week_trade) > 0:
        
        all_time_next_week_trade.append( {'week': e, 'next_week_trades': next_week_trade} )
    
    
    
    
    #print('--------------------------------------------------------------------------------------------', e )
    #print(next_week_trade)
    #print(len(next_week_trade))
    
    
    
if len(all_time_next_week_trade) == 0:
             
    raise Exception("no week passed the acceptable_signal_percent ")
        
        
    
print(len(all_time_next_week_trade))

In [ ]:
# creating a dataframe for nextweek trades and saving it in csv

for i in all_time_next_week_trade:
    
    current_week_data = i['next_week_trades']
    current_week_name = i['week']
    
    currency_name =[]
    trade_direction = []
    
    for j in current_week_data:
        
        currency_name.append(j['name'])
        trade_direction.append(j['direction'])
        
        
    
    
    
    
    next_week_trades_dataframe = pd.DataFrame()
    next_week_trades_dataframe['name']= currency_name
    next_week_trades_dataframe['direction'] = trade_direction
    
    
    print(next_week_trades_dataframe)

    next_week_trades_dataframe.to_csv("files/" + current_week_name + '/next_week_trades.csv')

In [ ]:
all_currency_thirty_minutes_data = {}

for z in all_currency_pairs:
    current_currency = z.replace('10080','')
    current_currency_df = pd.read_csv('../raw_30_minutes_data/' + current_currency + '30.csv', names=headers ) 
    
    all_currency_thirty_minutes_data[current_currency] = current_currency_df
    
    #print( current_currency, )
    
    
print(len(all_currency_thirty_minutes_data))




In [ ]:
qualified_weeks =[]

for k in all_time_next_week_trade:
    
    current_week_data = k['next_week_trades']
    current_week_date = k['week']
    
    current_week_date = datetime.strptime(current_week_date, '%Y.%m.%d')

    week_start_date  = current_week_date + timedelta(days=7)
    week_end_date = current_week_date + timedelta(days=12)
    
    current_week_date = current_week_date.strftime("%Y.%m.%d")
    week_start_date = week_start_date.strftime("%Y.%m.%d")
    week_end_date = week_end_date.strftime("%Y.%m.%d")      

    
    

    #print(current_week_date, week_start_date, week_end_date)
    
    qualified_currencies = 0
   
    for l in current_week_data:
        
        
        current_currency = l['name'] 
        
        current_currency_df = all_currency_thirty_minutes_data[l['name']].copy()
        
        
        
        
        current_currency_df = current_currency_df.loc[  (current_currency_df['date'] >= week_start_date)
                                                   
                                                                                 & 
                                                              (current_currency_df['date'] <= week_end_date)
                                                            ]  
    
        
        
        
        current_currency_df = current_currency_df.reset_index(drop=True )
        #print( week_start_date, current_currency, current_currency_df)
        
       
        current_currency_dataframe_row_ammount = len(current_currency_df.index)
        
        if  current_currency_dataframe_row_ammount < 238:
            
            print(current_currency,len(current_currency_df),week_start_date,'less than 238, skipping ',current_week_date)
            continue
            #raise Exception("lenght is lesss than 238")
            
            
        
        if  current_currency_dataframe_row_ammount != 240:
            
            if (current_currency_df['ignore'].tail(1).values == '22:30' ):
                
                
                
                last_row = current_currency_df.copy().tail(1)
                
                last_row.tail()['ignore'] = '23:00'
                
                current_currency_df = current_currency_df.append(last_row, ignore_index = True)
                
                
                last_row.tail()['ignore'] = '23:30'
                
                current_currency_df = current_currency_df.append(last_row, ignore_index = True)
                
                #print(current_currency, current_currency_df)
                
                if  len(current_currency_df.index) != 240:
              
                    print(current_currency, current_currency_df)
                    raise Exception("number of rows is still wrong after repair")
        
            else:
                
                print(current_currency,len(current_currency_df),week_start_date,"irrepairable,skipping ",current_week_date)
                continue
        
        if current_currency_df['ignore'].iloc[-1] != '23:30' :
        
            print(current_currency, current_currency_df)
            raise Exception("last row is not 23:30")
        
        
      
            
             
        
        
       
        current_currency_df.to_csv("files/" + k['week'] + '/' + current_currency +".csv", index=None)
        
        qualified_currencies = qualified_currencies +1
        
        
        
    if qualified_currencies == len(current_week_data):
            
        qualified_weeks.append(k)
        
    
   
        
        #print(current_currency_df)
        
        
        
    #print('-----------------------------------------------------------------------------------------------', week_start_date)


print(len(qualified_weeks))


In [ ]:
timeframe = []
trade_values = []
unique_time_frames = []

for m in qualified_weeks:

    
    
    current_week = m['week']
    current_week_data = m['next_week_trades']
    
    current_week_drawdown_df = pd.DataFrame()
    
    current_week_profit = pd.DataFrame()
  

    
    
    for index, n in enumerate(current_week_data):
        
        
        
        current_currency = n['name']
        current_currency_direction = n['direction']
        current_currency_df =  pd.read_csv("files/" + current_week + '/' + current_currency +".csv" ) 
        
        
        # adding timeframe to the maximal draw down df
        if index == 0:
            
            current_week_drawdown_df['date'] = current_currency_df['date']
            current_week_profit['date'] = current_currency_df['date']
           
            #saving unique_timeframes in a week to use later in the script
            if len(unique_time_frames) == 0:
                
                unique_time_frames = current_currency_df['ignore'].values
            
        
        
        #print(current_currency_df)
        maximal_draw_down =[]
        profit = []
        pips_corrector = 100000
            
        if current_currency.endswith('JPY'):
                
            pips_corrector = 1000
            
        
        
        
        for o in range(240):
                   
            if current_currency_direction == 'B':
                
                
                dd = current_currency_df.loc[o,'low'] - current_currency_df.loc[0,'open']
                p = current_currency_df.loc[o,'close'] - current_currency_df.loc[0,'open']
              
            if current_currency_direction == 'S':
                
                
                dd = current_currency_df.loc[0,'open'] - current_currency_df.loc[o,'high']
                p = current_currency_df.loc[0,'open'] - current_currency_df.loc[o,'close']
            
            
            maximal_draw_down.append(dd*pips_corrector)
            profit.append(p*pips_corrector)
                
        
        current_week_drawdown_df[current_currency] = maximal_draw_down
        current_week_profit[current_currency] = profit
            
    current_week_drawdown_df['empty'] = ['' for p in range(240)]      
    current_week_drawdown_df["sum"] = current_week_drawdown_df.drop(["empty", 'date'], axis=1, inplace=False).sum(axis=1)        
    
    current_week_drawdown_df.to_csv("files/" + m['week'] + '/' + 'maximal_draw_down.csv', index= None)
    
    
    
    current_week_profit['empty'] = ['' for p in range(240)]      
    current_week_profit["sum"] = current_week_profit.drop(["empty", 'date'], axis=1, inplace=False).sum(axis=1)        
    
    current_week_profit.to_csv("files/" + m['week'] + '/' + 'profit.csv', index= None)
    
    
    
    #print(current_week_drawdown_df)
        
    
    
    
    
    
    
    
    
    
    print('--------------------------------------------------------------------------------------------------', m)   
    
    
print(current_week_drawdown_df)

print(unique_time_frames)

In [ ]:
day_time_frames = []

for index, w in enumerate(unique_time_frames):
    
    if index >= 192: 
        day = 'FRI'  
        day_time_frames.append(day + ' ' + w)
        
    elif index >= 144: 
        day = 'THUR'  
        day_time_frames.append(day + ' ' + w)
        
    elif index >=96: 
        day = 'WED'  
        day_time_frames.append(day + ' ' + w)
        
    elif index >=48: 
        day = 'TUE'  
        day_time_frames.append(day + ' ' + w)
        
    else: 
        day = 'MON'  
        day_time_frames.append(day + ' ' + w)
    
   
    
print(len(day_time_frames))

In [ ]:
#combining all maximal drawdowns for all weeks
combine_week_drawdown_df = pd.DataFrame()

combine_week_drawdown_df['date'] = day_time_frames

for index, q in enumerate(qualified_weeks):
    
    current_week = q['week']
    current_week_dd_df = pd.read_csv("files/" + current_week + '/' + 'maximal_draw_down' +".csv" ) 
    #print(current_week_dd_df)
    
   
    combine_week_drawdown_df[current_week] = current_week_dd_df['sum']
    

combine_week_drawdown_df['empty'] = ['' for r in range(240)]      
combine_week_drawdown_df["sum"] = combine_week_drawdown_df.drop(["empty",'date' ], axis=1, inplace=False).sum(axis=1)        

combine_week_drawdown_df.to_csv("files/" + 'combine_week_drawdown_df.csv', index= None)

print(combine_week_drawdown_df)

In [ ]:
#combining all profits drawdowns for all weeks
combine_week_profit_df = pd.DataFrame()

combine_week_profit_df['date'] = day_time_frames

for index, q in enumerate(qualified_weeks):
    
    current_week = q['week']
    current_week_profit_df = pd.read_csv("files/" + current_week + '/' + 'profit' +".csv" ) 
    #print(current_week_dd_df)
    
   
    combine_week_profit_df[current_week] = current_week_profit_df['sum']
    

combine_week_profit_df['empty'] = ['' for r in range(240)]      
combine_week_profit_df["sum"] = combine_week_profit_df.drop(["empty",'date' ], axis=1, inplace=False).sum(axis=1)        

combine_week_profit_df.to_csv("files/" + 'combine_week_profit_df.csv', index= None)

print(combine_week_profit_df)

In [ ]:
#loading all profit and drawdown dataframe to memory to speed up processing

qualified_weeks_dfs=[]

for v in qualified_weeks:
    
    current_week = v['week']

    current_week_profit_df = pd.read_csv("files/" + current_week + '/profit.csv')
    current_week_drawdown_df = pd.read_csv("files/" + current_week + '/maximal_draw_down.csv')
    
    qualified_weeks_dfs.append( 
                                { 'week': current_week,
                                   'profit_df' : current_week_profit_df,
                                   'drawdown_df': current_week_drawdown_df
                                }
                                )
    

print(len(qualified_weeks_dfs))
    
    

In [ ]:
statistic_df = pd.DataFrame()

start_date_index_list =[]

cut_off_date_index_list =[]

draw_down_list = []

total_spread_list = []

profit_list=[]

profit_ratio_list=[]


combine_week_drawdown_df = pd.read_csv("files/" + 'combine_week_drawdown_df.csv')
combine_week_profit_df = pd.read_csv("files/" + 'combine_week_profit_df.csv')


#print(combine_week_drawdown_df)

start_date_index = [f for f in range(0, 239)]


draw_down = [f*-1 for f in range(100, 3000, 100)] 


total_spread = -200





for t in start_date_index:
    
    cut_off_date_index =   [f+1 for f in range( 1,(240-t)  ) ]
    
    #print(cut_off_date_index)
    
    for g in cut_off_date_index:
        
        
        for q in draw_down :
            
            
            strategy_profit = []
            
            current_maximal_drawdown = q + total_spread
            
            
            for s in qualified_weeks_dfs:
                
                current_week_calculated_profit = None
                
                current_drawdown_df = s['drawdown_df'].copy()
                current_drawdown_df = current_drawdown_df.iloc[t:]
                current_drawdown_df = current_drawdown_df.reset_index(drop=True)   
                current_drawdown_df = current_drawdown_df.iloc[:g]
                current_week_drawdown_list = current_drawdown_df['sum'].values
                
                

                current_profit_df = s['profit_df'].copy()
                current_profit_df = current_profit_df.iloc[t:]
                current_profit_df = current_profit_df.reset_index(drop=True)
                current_profit_df = current_profit_df.iloc[:g]
                current_week_profit_list = current_profit_df['sum'].values
                
                
                
                #print(current_week_profit_list)
                
            
                for r in range(len(current_week_drawdown_list)):
                    
                    
                    
                    current_drawdown = current_week_drawdown_list[r] - current_week_profit_list[0]
                    
                    current_profit = current_week_profit_list[r] - current_week_profit_list[0]
                    
                    
                    if r == 0:
                        #skipping the first row
                        continue
                    
                    
                    if current_drawdown <= q:
                        
                        
                        current_week_calculated_profit = current_maximal_drawdown
                    
                        break
                        
                    
                    
                if current_week_calculated_profit == None:
                    
                    current_week_calculated_profit = current_profit
                    
                
                strategy_profit.append(current_week_calculated_profit)
                
            total_profit = sum(strategy_profit)
            
            profit_ratio = total_profit/(10*current_maximal_drawdown*-1)
            
            
            #print( 'start:', t+1, '  end:', g, '  drawdown:',q , '  profit:', total_profit, 'profit_ratio:',profit_ratio )
                
            start_date_index_list.append(t+1)

            cut_off_date_index_list.append(g)

            draw_down_list.append(q)

            total_spread_list.append(total_spread)

            profit_list.append(total_profit)

            profit_ratio_list.append(profit_ratio)
            
    print(t)

In [ ]:
statistic_df['start_date_index'] = start_date_index_list


statistic_df['cut_off_date_index'] = cut_off_date_index_list

statistic_df['total_spread'] = total_spread_list

statistic_df['draw_down'] = draw_down_list





statistic_df['profit'] = profit_list

statistic_df['profit_ratio'] = profit_ratio_list





print(statistic_df)
statistic_df.to_csv('files/statistic_df.csv')